In [1]:
import json

with open("../data/processed/scrapped.json", "rb") as f:
    data = json.load(f)



In [4]:
data[0].keys()

dict_keys(['_id', 'site', 'domain', 'title', 'content', 'date', 'label', 'searched', 'alike_news', 'content_keywords', 'title_keywords'])

In [2]:
import json

with open("./domains.json", "rb") as f:
    domains = json.load(f)


In [3]:
minimun_ts: float = 999
maximun_ts: float = 0

ts_list = []

for key, value in domains.items():
    if float(value["trust_score"]) < minimun_ts:
        minimun_ts = float(value["trust_score"])
    if float(value["trust_score"]) > maximun_ts:
        maximun_ts = float(value["trust_score"])
    ts_list.append(float(value["trust_score"]))

print(f"Minimun trust score: {minimun_ts}")
print(f"Maximun trust score: {maximun_ts}")

Minimun trust score: -0.1
Maximun trust score: 6.7


In [4]:
minimun_score: float = 999
maximun_score: float = 0
score_list = []

for key, value in domains.items():
    if float(value["score"]) < minimun_score:
        minimun_score = float(value["score"])
    if float(value["score"]) > maximun_score:
        maximun_score = float(value["score"])
    score_list.append(float(value["score"]))

print(f"Minimun score: {minimun_score}")
print(f"Maximun score: {maximun_score}")

Minimun score: 1.0
Maximun score: 73.0


In [5]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler


ts_scaller = MinMaxScaler()
score_scaller = MinMaxScaler()

ts_scallered = ts_scaller.fit_transform(np.array(ts_list).reshape(-1, 1))
score_scallered = score_scaller.fit_transform(np.array(score_list).reshape(-1, 1))


In [6]:
# lowercase keys
domains_lc = {}
for key, value in domains.items():
    domains_lc[key.lower()] = value


```json
{
    "_id": "",
    "site": "",
    "domain": "",
    "title": "",
    "content": "",
    "date": "",
    "label": 1,
    "searched": 0,
    "alike_news": [],
    "content_keywords": [
        {
            "url": "",
            "title": "",
            "text": "",
            "is_similar": 0,
            "slug": "",
            "meta": "",
        },
        {}
    ],
    "title_keywords": [],
}
```

In [7]:
def remove_www(domain: str):
    domain = domain.replace("feed", "")
    if domain.startswith("https://"):
        domain = domain[8:]

    if domain.startswith("http://"):
        domain = domain[7:]

    if domain.startswith("www."):
        return domain[4:-1]
    return domain[:-1]


def extract_domain_from_url(url: str):
    return url.split("/")[2]


In [12]:
import statistics

for entry in data:
    site = entry["site"]
    if site not in domains_lc.keys():
        site = remove_www(site)
    try:
        values = [score_scaller.transform([[domains_lc[site]["score"]]])[0][0]]
    except Exception:
        values = [score_scaller.transform([[domains_lc["www." + site]["score"]]])[0][0]]
    try:
        weights = [1 - ts_scaller.transform([[domains_lc[site]["trust_score"]]])[0][0]]
    except Exception:
        weights = [1 - ts_scaller.transform([[domains_lc["www." + site]["trust_score"]]])[0][0]]

    if site == "timesnewroman.ro":
        entry["label"] = 0
        continue

    for news in entry["alike_news"]:
        if news is not None:
            if "has_similar_content" in news.keys() or "has_similar_title" in news.keys():
                if news["has_similar_content"] or news["has_similar_title"]:
                    if "wikipedia" in news["url"]:
                        values.append(1)
                        weights.append(1)
                    else:
                        site = extract_domain_from_url(news["url"])
                        if site not in domains_lc.keys():
                            site = remove_www(site)
                    try:
                        values.append(score_scaller.transform([[domains_lc[site]["score"]]])[0][0])
                        weights.append(
                            1 - ts_scaller.transform([[domains_lc[site]["trust_score"]]])[0][0]
                        )
                    except KeyError:
                        values.append(0.5)
                        weights.append(0.5)

    entry["label"] = statistics.harmonic_mean(values, weights=weights)


In [25]:
for entry in data:
    entry["label"] = round(entry["label"], 2)

```json
{
    "_id": "",
    "site": "",
    "domain": "",
    "title": "",
    "content": "",
    "date": "",
    "label": 1,
    "searched": 0,
    "alike_news": [
        {
            "url": "",
            "title": "",
            "text": "",
            "is_similar": 0,
            "slug": "",
            "meta": "",
        },
        {}
    ],
    "content_keywords": [],
    "title_keywords": [],
}
```

In [13]:
for entry in data:
    del entry["searched"]
    del entry["alike_news"]
    del entry["content_keywords"]
    del entry["title_keywords"]

In [27]:
import pandas as pd

df = pd.DataFrame(data)

In [28]:
df["date"] = [pd.Timestamp(x["$date"]) for x in df["date"]]

In [29]:
df["_id"] = [x["$oid"] for x in df["_id"]]

In [30]:
df[df["site"] == "https://www.timesnewroman.ro/"]["label"] = 0

/tmp/ipykernel_28054/1263213858.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [31]:
df.to_parquet("../data/raw/scrapped_data/celery_scrapped_data/scrappedv2.gzip", compression='gzip')

In [19]:
df.site.unique()

array(['https://www.digi24.ro/', 'https://www.aktual24.ro/',
       'https://www.timesnewroman.ro/', 'https://caplimpede.ro/',
       'https://www.activenews.ro/', 'https://www.biziday.ro/',
       'https://www.biziday.ro/feed', 'https://infoalert.ro/'],
      dtype=object)

In [22]:
df.head()

,_id,site,domain,title,content,date,label
0,62740edcedb541967dd03cb6,https://www.digi24.ro/,stiri/externe,"Bursele asiatice în declin, petrolul în cădere...",Bursele asiatice înregistrau scăderi pe linie ...,2020-03-12 09:31:00+00:00,0.763889
1,62740edcedb541967dd03cb7,https://www.digi24.ro/,stiri/externe,Ministrul britanic al sănătății: Nu vom ezita ...,"Matt Hancock, ministrul britanic al sănătății,...",2020-07-30 11:46:00+00:00,0.763889
2,62740edcedb541967dd03cbd,https://www.digi24.ro/,stiri/actualitate,Cum a pierdut PSD majoritatea în Biroul Perman...,Absența senatorului PSD Ion Ganea de la întrun...,2020-10-27 18:58:00+00:00,0.763889
3,62740edcedb541967dd03cb5,https://www.digi24.ro/,stiri/externe,Șase turiști faliți au fost salvați după 25 de...,Șase turiști care locuiau într-o peșteră din I...,2020-04-21 11:54:00+00:00,0.763889
4,62740edcedb541967dd03cbe,https://www.digi24.ro/,stiri/actualitate,Coronavirus. Pensionarul de la MAI care a minț...,Bărbatul de 60 de ani infectat cu coronavirus ...,2020-03-11 13:44:00+00:00,0.763889
